In [1]:
import random

from cachetools import cached, LFUCache, LRUCache, FIFOCache, RRCache

from mlfucache import MLFUCache
from benchmark.beladycache import BeladyCache

In [2]:
n = 10000
cbase = n // 10
size = cbase // 10

In [3]:
s = 1.1

HNs = sum([k ** -s for k in range(1, cbase + 1)])


def cdf(k, s):
    return sum([k ** -s / HNs])


keys = range(1, cbase + 1)
weights = [cdf(k, s) for k in keys]
requests = random.choices(keys, weights=weights, k=n)

In [4]:
class MyCache(MLFUCache):
    def popitem(self):
        key, value = super().popitem()
        print('Key "%s" evicted with value "%s"' % (key, value))
        return key, value


In [5]:
for cache in [RRCache(maxsize=size), FIFOCache(maxsize=size), LRUCache(maxsize=size), LFUCache(maxsize=size),
              MLFUCache(maxsize=size), BeladyCache(maxsize=size, future=requests)]:

    @cached(cache=cache, key=lambda k: k, info=True)
    def storage(key) -> None:
        return None


    for key in requests:
        storage(key)

    print(f"{cache.__class__.__name__}, {storage.cache_info()}")


RRCache, CacheInfo(hits=6148, misses=3852, maxsize=100, currsize=100)
FIFOCache, CacheInfo(hits=6111, misses=3889, maxsize=100, currsize=100)
LRUCache, CacheInfo(hits=6629, misses=3371, maxsize=100, currsize=100)
LFUCache, CacheInfo(hits=7239, misses=2761, maxsize=100, currsize=100)
MLFUCache, CacheInfo(hits=7425, misses=2575, maxsize=100, currsize=100)
BeladyCache, CacheInfo(hits=6811, misses=3189, maxsize=100, currsize=100)
